# Test 3: Table Handling with Layout Parser

**Goal:** Extract tables and convert them to narrative paragraphs using LLM

**What this test does:**
- Detects tables using Layout Parser (superior table detection)
- Extracts table data into structured format
- Converts tables to markdown format
- Uses LLM to generate narrative descriptions of tables
- Saves both markdown and narrative versions

**Why convert tables to narratives?**
- Better for semantic search and taxonomy matching
- Makes textbook content more accessible
- Preserves table information in readable format
- Layout Parser provides excellent table boundary detection


## Step 1: Install Dependencies

Run this cell to install required packages for Google Colab.


In [ ]:
%pip install -q google-cloud-documentai python-dotenv openai anthropic pdf2image Pillow
print("✅ All dependencies installed!")


## Step 2: Upload Credentials

Upload your Google Cloud service account JSON file.


In [ ]:
from google.colab import files
import json
import os

uploaded = files.upload()
creds_filename = list(uploaded.keys())[0]
credentials_content = json.loads(uploaded[creds_filename].decode('utf-8'))

with open('docai-credentials.json', 'w') as f:
    json.dump(credentials_content, f)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'docai-credentials.json'
print(f"✅ Credentials saved")


## Step 3: Configure Environment

Set your Google Cloud project ID and Layout Parser processor ID.


In [ ]:
# Configuration - UPDATE THESE VALUES
DOCAI_PROJECT_ID = "your-project-id-here"
DOCAI_PROCESSOR_ID = "your-processor-id-here"
DOCAI_LOCATION = "us"

# LLM Configuration for table conversion
OPENAI_API_KEY = "sk-your-openai-key-here"  # Or use ANTHROPIC_API_KEY
LLM_PROVIDER = "openai"  # or "anthropic"
LLM_MODEL = "gpt-4o"

os.environ['DOCAI_PROJECT_ID'] = DOCAI_PROJECT_ID
os.environ['DOCAI_PROCESSOR_ID'] = DOCAI_PROCESSOR_ID
os.environ['DOCAI_LOCATION'] = DOCAI_LOCATION
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['LLM_PROVIDER'] = LLM_PROVIDER
os.environ['LLM_MODEL'] = LLM_MODEL

print(f"✅ Configuration set")


## Step 4: Clone Repository and Load Utils

Clone the repository to access utility functions.


In [ ]:
!git clone https://github.com/abhii-01/python-automation.git
%cd python-automation

import sys
from pathlib import Path
sys.path.append(str(Path.cwd()))

from utils.docai_client import get_client_from_env
from utils.table_converter import (
    extract_table_data,
    table_to_markdown,
    table_to_narrative,
    detect_table_type
)

print("✅ Repository cloned and utilities loaded")


## Step 5: Verify Setup

Test connection to Google Document AI.


In [ ]:
print("🔍 Verifying Document AI setup...\n")

try:
    client = get_client_from_env()
    client.verify_setup()
    print("\n✅ Setup verified! Ready to process documents.")
except Exception as e:
    print(f"\n❌ Setup verification failed: {e}")
    print("\nPlease check:")
    print("  1. Credentials file is valid")
    print("  2. Project ID is correct")
    print("  3. Processor ID is correct")
    print("  4. Layout Parser processor exists and is enabled")


## Step 6: Upload PDF for Testing

Upload your PDF file with tables to process.


In [ ]:
print("📤 Please upload your PDF file (should contain tables)...")
uploaded_pdfs = files.upload()

pdf_filename = list(uploaded_pdfs.keys())[0]
pdf_path = pdf_filename

print(f"✅ PDF uploaded: {pdf_filename}")
print(f"   Size: {len(uploaded_pdfs[pdf_filename]) / 1024:.1f} KB")


## Step 7: Process Document with Layout Parser

Process the PDF to detect tables.


In [ ]:
print(f"{'='*60}")
print("TEST 3: TABLE HANDLING")
print(f"{'='*60}\n")

print(f"📄 Processing PDF with Layout Parser: {pdf_path}")
document = client.process_document(pdf_path)

print(f"✅ Document processed!")
print(f"   Total pages: {len(document.pages)}")


## Step 8: Define Helper Function for Table Extraction


In [ ]:
client = get_client_from_env()
client.verify_setup()
print("\n✅ Ready to process tables!")


In [ ]:
print("📤 Please upload a PDF with tables...")
uploaded_pdfs = files.upload()

pdf_filename = list(uploaded_pdfs.keys())[0]
pdf_path = pdf_filename

print(f"✅ PDF uploaded: {pdf_filename}")


## Process Document and Extract Tables


In [ ]:
print(f"{'='*60}")
print("TEST 3: TABLE HANDLING")
print(f"{'='*60}\n")

print(f"📄 Processing PDF with Layout Parser: {pdf_path}")
document = client.process_document(pdf_path)

print(f"✅ Document processed!")
print(f"   Total pages: {len(document.pages)}")


## Define Helper Function for Table Extraction


In [ ]:
def extract_simple_table(table, full_text):
    """
    Simplified table extraction
    Returns 2D list of cell values
    """
    cells_dict = {}
    max_row = 0
    max_col = 0
    
    # Extract all cells
    all_cells = []
    if hasattr(table, 'header_rows'):
        for row in table.header_rows:
            all_cells.extend(row.cells)
    if hasattr(table, 'body_rows'):
        for row in table.body_rows:
            all_cells.extend(row.cells)
    
    # Build cell grid
    for cell in all_cells:
        if not hasattr(cell, 'layout') or not cell.layout.text_anchor:
            continue
        
        # Get cell position
        row_idx = cell.layout.table_row_index if hasattr(cell.layout, 'table_row_index') else 0
        col_idx = cell.layout.table_col_index if hasattr(cell.layout, 'table_col_index') else 0
        
        # Get cell text
        text_parts = []
        for segment in cell.layout.text_anchor.text_segments:
            text = full_text[segment.start_index:segment.end_index]
            text_parts.append(text)
        
        cell_text = " ".join(text_parts).strip()
        
        cells_dict[(row_idx, col_idx)] = cell_text
        max_row = max(max_row, row_idx)
        max_col = max(max_col, col_idx)
    
    # Convert to 2D list
    if not cells_dict:
        return []
    
    table_data = []
    for row in range(max_row + 1):
        row_data = []
        for col in range(max_col + 1):
            cell_text = cells_dict.get((row, col), "")
            row_data.append(cell_text)
        table_data.append(row_data)
    
    return table_data

print("✅ Helper function defined")


## Extract and Process Tables


In [ ]:
print("\n📊 Processing tables...\n")

table_results = []
table_count = 0

for page_num, page in enumerate(document.pages, 1):
    if not hasattr(page, 'tables') or not page.tables:
        continue
    
    print(f"  Page {page_num}: Found {len(page.tables)} table(s)")
    
    for table_idx, table in enumerate(page.tables):
        table_count += 1
        print(f"\n  Processing table {table_count}...")
        
        # Extract table data
        table_data = extract_simple_table(table, document.text)
        
        if not table_data:
            print(f"    ⚠️  Could not extract table data")
            continue
        
        # Convert to markdown
        markdown = table_to_markdown(table_data)
        print(f"    ✅ Converted to markdown ({len(table_data)} rows)")
        
        # Detect table type
        table_type = detect_table_type(table_data)
        print(f"    🔍 Detected type: {table_type}")
        
        # Convert to narrative
        print(f"    🤖 Converting to narrative paragraph...")
        try:
            narrative = table_to_narrative(markdown, method=table_type)
            print(f"    ✅ Generated narrative ({len(narrative)} chars)")
        except Exception as e:
            print(f"    ⚠️  LLM conversion failed: {e}")
            narrative = f"[Table conversion failed: {e}]"
        
        # Store result
        result = {
            "table_id": f"table_{table_count}",
            "page": page_num,
            "table_index": table_idx,
            "rows": len(table_data),
            "columns": len(table_data[0]) if table_data else 0,
            "detected_type": table_type,
            "markdown": markdown,
            "narrative": narrative,
            "original_data": table_data
        }
        
        table_results.append(result)

print(f"\n✅ Processed {table_count} tables")


In [ ]:
if table_results:
    example = table_results[0]
    
    print(f"\n{'='*60}")
    print(f"EXAMPLE: Table 1 (Page {example['page']})")
    print(f"{'='*60}\n")
    
    print(f"📊 Original Markdown:")
    print("-" * 60)
    print(example['markdown'][:500])
    if len(example['markdown']) > 500:
        print("...")
    print("-" * 60)
    
    print(f"\n📝 Generated Narrative:")
    print("-" * 60)
    print(example['narrative'])
    print("-" * 60)
else:
    print("\n⚠️  No tables found in this document")


## Save Results


In [ ]:
# Build final results
results = {
    "pdf_file": Path(pdf_path).name,
    "total_tables": table_count,
    "tables": table_results
}

# Save to JSON
output_path = "test3_tables.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"\n💾 Results saved to: {output_path}")
print(f"\n📊 Summary:")
print(f"  Total tables found: {table_count}")
print(f"  Successfully converted: {len(table_results)}")

# Download results
files.download(output_path)
print(f"\n✅ Test 3 complete! Results downloaded.")
